### Model notes:
- Only 60 mo product
- Use SMOTE to upsample minority class and gridsearch

#### Results:


In [132]:
#Installing imblearn
#!pip install -U imbalanced-learn

In [133]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 50)

In [134]:
%%time
df = pd.read_parquet('../data/approved.parquet', engine='fastparquet')

Wall time: 2.83 s


In [135]:
df.sample(5)

,addr_state,annual_inc,application_type,disbursement_method,dti,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,grade,home_ownership,initial_list_status,installment,int_rate,issue_d,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,purpose,sub_grade,term,verification_status,zip_code,loan_status
id,,,,,,,,,,,,,,,,,,,,,,,,,,
31257099,AL,58000.0,Individual,Cash,21.54,2001-08-01,2,Outbound Associate,699.0,695.0,C,MORTGAGE,w,310.51,14.49,2014-10-01,13200.0,10.0,0.0,0.0,debt_consolidation,C4,60,Source Verified,350xx,1
28873688,MI,80000.0,Individual,Cash,13.85,1998-12-01,1,Owner,669.0,665.0,C,MORTGAGE,w,173.31,14.99,2014-10-01,5000.0,8.0,1.0,0.0,small_business,C5,36,Source Verified,480xx,1
37711991,FL,45000.0,Individual,Cash,13.47,2008-08-01,9,Customerservice,664.0,660.0,D,RENT,f,495.60,16.49,2014-12-01,14000.0,20.0,0.0,0.0,credit_card,D3,36,Source Verified,331xx,0
60852209,AR,83500.0,Individual,Cash,9.62,2004-02-01,3,Director of IT,679.0,675.0,C,RENT,f,275.96,14.65,2015-09-01,8000.0,13.0,0.0,0.0,other,C5,36,Verified,727xx,1
22491532,CA,60000.0,Individual,Cash,14.34,2005-02-01,10,Program Manager,724.0,720.0,B,MORTGAGE,f,323.38,10.15,2014-08-01,10000.0,10.0,0.0,0.0,credit_card,B2,36,Source Verified,917xx,1


### Feature Engineering

In [136]:
df['days_cr_line']=df['issue_d']-df['earliest_cr_line']
#convert to int (https://www.codegrepper.com/code-examples/python/pandas+timedelta64+ns+to+float)
df['days_cr_line'] = (df['days_cr_line'] / np.timedelta64(1,'D')).astype(int)

### Convert Ordinals

In [137]:
# sub_grade
df.sub_grade.replace({'A1':1, 'A2':2, 'A3':3, 'A4':4, 'A5':5, 'B1':6, 'B2':7, 
                      'B3':8, 'B4':9, 'B5':10, 'C1':11, 'C2':12, 'C3':13, 'C4':14, 
                      'C5':15, 'D1':16, 'D2':17, 'D3':18, 'D4':19, 'D5':20, 'E1':21, 
                      'E2':22, 'E3':23, 'E4':24, 'E5':25}, inplace=True)

# disbursement_method
df.disbursement_method.replace({'Cash':0,'DirectPay':1},inplace=True)

# verification_status
df.verification_status.replace({'Source Verified':'Verified'},inplace=True)
df.verification_status.replace({'Not Verified':0,'Verified':1}, inplace=True)

# initial_list_status
df.initial_list_status.replace({'w':0,'f':1}, inplace=True)

# application_type
df.application_type.replace({'Individual':0,'Joint App':1}, inplace=True)

### Dummification

In [138]:
dummies=['purpose','addr_state','home_ownership']

for d in dummies:
    temp = pd.get_dummies(df[d], prefix=d, prefix_sep='__')
    temp = temp.drop(d+'__'+str(df[d].mode()[0]), axis=1)
    #drop the most common one (mode), making it the base case
    df = pd.concat([df.drop(d, axis=1),temp], axis=1)
    #merge back with main df

### Filter for years < 2013 because this is just the 60mo product

In [139]:
df = df.loc[df['issue_d'].dt.year.between(2007,2013)]

### Drop Columns

In [140]:
df.drop(['emp_title','zip_code','grade','issue_d','earliest_cr_line'],axis=1, inplace=True)
#removed 'issue_d'

### Drop Rows

In [141]:
df.drop(df[df['emp_length']==-1].index, inplace=True)

### Drop the 60month product

In [142]:
df = df.loc[df['term']==60]
df.drop('term', axis=1, inplace=True)

# Modelling

### create target and features

In [143]:
target = df['loan_status']

In [144]:
target.value_counts()

1    35614
0    10901
Name: loan_status, dtype: int64

In [145]:
features = df.drop(['loan_status'], axis=1)

### note the reduced train_size of .10 = ~120,000 obs

In [146]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split, KFold

Xtrain, Xtest, ytrain, ytest = train_test_split(features, target, train_size=.10 )
logr = LogisticRegression()

In [147]:
# logr.get_params()

### SMOTE to oversample minority class

In [148]:
%%time
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 101)
X_oversample, y_oversample = smote.fit_resample(Xtrain, ytrain)

Wall time: 100 ms


In [149]:
print(y_oversample.value_counts())
print(ytest.value_counts())

1    3540
0    3540
Name: loan_status, dtype: int64
1    32074
0     9790
Name: loan_status, dtype: int64


In [150]:
X_oversample, y_oversample = smote.fit_resample(Xtrain, ytrain)

### gridsearch

In [151]:
Cs = [0.0001,0.001,0.01,1,10,100,1000,10000]
params = [{'C':Cs, 'penalty':['l1','l2']}]
grid_logr = GridSearchCV(estimator=logr, param_grid=params, cv=3)

In [152]:
%%time
grid_logr.fit(X_oversample, y_oversample)

Wall time: 1.54 s


GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid=[{'C': [0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000],
                          'penalty': ['l1', 'l2']}])

In [153]:
print(grid_logr.best_params_)
print(grid_logr.best_score_)

{'C': 1, 'penalty': 'l2'}
0.5919491525423729


In [154]:
best_model = grid_logr.best_estimator_
best_model.score(Xtest,ytest)

0.5713739728645136

In [155]:
coefs = pd.DataFrame(abs(best_model.coef_).T, index=features.columns)

In [156]:
coefs.sort_values(0, ascending=False).head(10)

,0
sub_grade,0.044037
int_rate,0.041673
emp_length,0.036040
dti,0.011708
home_ownership__RENT,0.009029
purpose__credit_card,0.008970
addr_state__TX,0.004831
home_ownership__OWN,0.004005
initial_list_status,0.003796
open_acc,0.003788


### Confusion Matrix

In [157]:
from sklearn.metrics import confusion_matrix

In [158]:
cmat = confusion_matrix(ytest,best_model.predict(Xtest))

#Label the confusion matrix
pd.DataFrame(cmat, columns = [f'Pred_{label}' for label in best_model.classes_],
             index = [f'True_{label}' for label in best_model.classes_])


,Pred_0,Pred_1
True_0,5761,4029
True_1,13915,18159


### Best model is one that strikes good balance of a high "pred_1 precision" and a high "loan approval rate"

In [159]:
cmat=cmat
print(f'loan approval rate: {sum(cmat[:,1])/sum(sum(cmat)):.2f}')
print(f'precision pred_1: {cmat[1][1]/sum(cmat[:,1]):.2f}')
print(f'recall pred_1: {cmat[1][1]/sum(cmat[1,:]):.2f}')

loan approval rate: 0.53
precision pred_1: 0.82
recall pred_1: 0.57


In [160]:
from sklearn.metrics import classification_report
print(classification_report(ytest, best_model.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.29      0.59      0.39      9790
           1       0.82      0.57      0.67     32074

    accuracy                           0.57     41864
   macro avg       0.56      0.58      0.53     41864
weighted avg       0.70      0.57      0.60     41864



### change the threshold to improve the FP score

In [161]:
probs_positive_class = best_model.predict_proba(Xtest)[:, 1]
prediction = probs_positive_class > .6

In [162]:
cmat2 = confusion_matrix(ytest, prediction, labels=[0, 1])
pd.DataFrame(cmat2, columns = [f'Pred_{label}' for label in best_model.classes_],
             index = [f'True_{label}' for label in best_model.classes_])

,Pred_0,Pred_1
True_0,8208,1582
True_1,22975,9099


In [163]:
from sklearn.metrics import classification_report
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.26      0.84      0.40      9790
           1       0.85      0.28      0.43     32074

    accuracy                           0.41     41864
   macro avg       0.56      0.56      0.41     41864
weighted avg       0.71      0.41      0.42     41864



In [164]:
cmat=cmat2
print(f'loan approval rate: {sum(cmat[:,1])/sum(sum(cmat)):.2f}')
print(f'precision pred_1: {cmat[1][1]/sum(cmat[:,1]):.2f}')
print(f'recall pred_1: {cmat[1][1]/sum(cmat[1,:]):.2f}')

loan approval rate: 0.26
precision pred_1: 0.85
recall pred_1: 0.28


## ROC Curve

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
# predict probabilities
yhat = logr.predict_proba(Xtest)

In [ ]:
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

In [ ]:
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

In [ ]:
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

In [ ]:
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, prediction)

In [ ]:
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='Logistic')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

### SGD classifier

### Save model in pickle

In [ ]:
filename = 'grid_logr_model.sav'
pickle.dump(grid_logr, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(Xtest, ytest)
print(result)